In [1]:
"""
Course_Recommender 알고리즘
1. Word2Vec으로 산책로 keyword 학습 진행 (이게 끝임)
"""
"""
db에 적용시킬 알고리즘
1. 유저 A의 finish_courses 테이블에서 완주한 course의 id값을 가져온다.
2. 이 id 값을 course_types 테이블에서 각각의 tag들을 찾는다.
3. 각각의 course의 tag를 다 Word2Vec으로 벡터 변환
4. 변환 시킨 벡터를 다 더하고 course의 개수로 나눔 -> 이것이 유저 A의 키워드 벡터
5. 3에서 변환한 테이블에서 A가 이용했던 id 는 제거
6. 유저 A의 키워드 벡터와 3에서 변환한 테이블에서 각 course와 코사인 유사도 값 계산
7. 코사인 유사도 점수가 높은 course id를 A한테 5개 뽑아준다.
"""

'\ndb에 적용시킬 알고리즘\n1. 유저 A의 finish_courses 테이블에서 완주한 course의 id값을 가져온다.\n2. 이 id 값을 course_types 테이블에서 각각의 tag들을 찾는다.\n3. 각각의 course의 tag를 다 Word2Vec으로 벡터 변환\n4. 변환 시킨 벡터를 다 더하고 course의 개수로 나눔 -> 이것이 유저 A의 키워드 벡터\n5. 3에서 변환한 테이블에서 A가 이용했던 id 는 제거\n6. 유저 A의 키워드 벡터와 3에서 변환한 테이블에서 각 course와 코사인 유사도 값 계산\n7. 코사인 유사도 점수가 높은 course id를 A한테 5개 뽑아준다.\n'

In [2]:
# Word2Vec으로 keyword 학습
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


In [25]:
def tokenizer(text):
    temp = text.split(" ")
    temp_list = [[single] for single in temp]
    return temp_list

In [15]:
keyword_raw = """힐링 스타벅스 자연 오솔길 도심 출근길 퇴근길 점심시간 스트레스해소
한강 공원 성수 강아지 바다 해안가 러닝 맛집 카페 영화 문화 사색
핫플 서울숲 경복궁 한옥마을 문화재 고양이 개울가 계곡 들판 산 동산 야경 노을 숲길
강서구 양천구 구로구 영등포구 금천구 동작구 관악구 서초구 강남구 송파구 강동구
은평구 서대문구 마포구 용산구 중구 종로구 도봉구 강북구 성북구 동대문구 성동구 노원구
중랑구 광진구"""
keyword = keyword_raw.replace("\n", " ")
keyword = keyword.split(" ")
keyword_list = [[single] for single in keyword]
print(keyword_list)
model = Word2Vec(sentences = keyword_list, vector_size=20, window=1, min_count=1, workers=4)
model.save("word2vec.model")

[['힐링'], ['스타벅스'], ['자연'], ['오솔길'], ['도심'], ['출근길'], ['퇴근길'], ['점심시간'], ['스트레스해소'], ['한강'], ['공원'], ['성수'], ['강아지'], ['바다'], ['해안가'], ['러닝'], ['맛집'], ['카페'], ['영화'], ['문화'], ['사색'], ['핫플'], ['서울숲'], ['경복궁'], ['한옥마을'], ['문화재'], ['고양이'], ['개울가'], ['계곡'], ['들판'], ['산'], ['동산'], ['야경'], ['노을'], ['숲길'], ['강서구'], ['양천구'], ['구로구'], ['영등포구'], ['금천구'], ['동작구'], ['관악구'], ['서초구'], ['강남구'], ['송파구'], ['강동구'], ['은평구'], ['서대문구'], ['마포구'], ['용산구'], ['중구'], ['종로구'], ['도봉구'], ['강북구'], ['성북구'], ['동대문구'], ['성동구'], ['노원구'], ['중랑구'], ['광진구']]


In [29]:
print(model.wv.vectors.shape)
print(type(model.wv['힐링']))

(60, 20)
<class 'numpy.ndarray'>


In [41]:
import numpy as np
model = Word2Vec.load("word2vec.model")
example = "힐링 스타벅스 자연"
example_list = tokenizer(example)
course_vector = np.zeros((1,20))
for i in example_list :
    print(model.wv[i])
    course_vector = course_vector + model.wv[i]
course_vector = course_vector / len(example_list)
print("예시 산책로의 vector :", course_vector)


[[-0.01030495  0.04866779 -0.03428295 -0.0109586   0.035005   -0.00027875
  -0.03147484 -0.03196763  0.04470197  0.03214788  0.02386796 -0.01631024
  -0.0463381   0.01893444  0.03580275 -0.02816445 -0.03932507 -0.0148637
  -0.02465949 -0.01157556]]
[[ 0.04197962 -0.00340668  0.04572205 -0.04079111  0.01871542  0.01317521
   0.00371357  0.01163838 -0.03734547 -0.04679187  0.01177288  0.03074228
   0.03992844  0.02867947 -0.00388668  0.04153083 -0.04668157  0.01703066
   0.00133377  0.01928622]]
[[ 0.04776092 -0.03678212 -0.03635194 -0.01132694 -0.0038928  -0.01608052
  -0.00296293  0.03744411 -0.00348759 -0.0081247   0.013722   -0.0417955
   0.03927902  0.04268052 -0.04792044  0.01223133  0.04952485 -0.03832902
  -0.03483459 -0.03868259]]
예시 산책로의 vector : [[ 0.02647853  0.00282633 -0.00830428 -0.02102555  0.01660921 -0.00106135
  -0.0102414   0.00570495  0.00128964 -0.00758957  0.01645428 -0.00912116
   0.01095646  0.03009814 -0.00533479  0.00853257 -0.01216059 -0.01205402
  -0.01938677